In [1]:
#import cx_Oracle
import pandas as pd
import sqlalchemy
from sqlalchemy.exc import SQLAlchemyError

<h1>Pobieranie danych z bazy</h1>

In [32]:
data = {}

dbtables = ['MEB_DGM', 'MEB_DMC', 'MEB_GROB', 'MEB_KO', 'MEB_KO_DGM', 'MEB_KO_RODZAJ', 'MEB_KO_STREFA', 'MEB_KS', 'ONI_CIRCUITS']
querys = [
#Tutaj występują duplikaty, ale jeszcze nie zdecydowaliśmy co z nimi zrobić
"""
SELECT *
FROM (
  SELECT
    t.*,
    ROW_NUMBER() OVER (PARTITION BY DMC ORDER BY ID DESC) AS rn
  FROM
    Z3DMC.MEB_DGM t
) subquery
WHERE rn = 1
""",
#Tutaj występują duplikaty, ale jeszcze nie zdecydowaliśmy co z nimi zrobić, usunąłem tylko nulle i anomalię
#Po wykonaniu LEFT JOIN liczba rekordów się zwiększyła, trzeba usunąć duplikaty z DMC_CASTING ~ LO
# Zapytanie dodatkowo nie pobiera wartości null z DMC_CASTING
"""
SELECT *
FROM Z3DMC.MEB_DMC
WHERE (DMC <> 'null') AND (DMC <> 'NIECZYTELNY') AND (DMC_CASTING IS NOT NULL)
""",
"""
SELECT *
FROM (
  SELECT
    t.*,
    ROW_NUMBER() OVER (PARTITION BY ID_DMC ORDER BY ID_MEB_GROB DESC) AS rn
  FROM
    Z3DMC.MEB_GROB t
) subquery
WHERE rn = 1
""",
"""
SELECT *
FROM (
  SELECT
    t.*,
    ROW_NUMBER() OVER (PARTITION BY ID_DMC ORDER BY ID_KO DESC) AS rn
  FROM
    Z3DMC.MEB_KO t
) subquery
WHERE rn = 1
""",
"""
SELECT *
FROM (
  SELECT
    t.*,
    ROW_NUMBER() OVER (PARTITION BY ID_DMC ORDER BY ID_KO DESC) AS rn
  FROM
    Z3DMC.MEB_KO_DGM t
  WHERE ID_DMC <> 0
) subquery
WHERE rn = 1
""",
#Tabela słownik, niczego nie usuwamy, wczytuję całą tabelę
"""
SELECT *
FROM Z3DMC.MEB_KO_RODZAJ
""",
#Tabela słownik, niczego nie usuwamy, wczytuję całą tabelę
"""
SELECT *
FROM Z3DMC.MEB_KO_STREFA
""",
"""
SELECT *
FROM (
  SELECT
    t.*,
    ROW_NUMBER() OVER (PARTITION BY ID_DMC ORDER BY ID_KS DESC) AS rn
  FROM
    Z3DMC.MEB_KS t
) subquery
WHERE rn = 1
""",
#Tabela ONI_CIRCUITS posiada wiele duplikatów, a powinno być ich do 28 per jeden ID_DMC, dlatego wrzucam całą tabelę i przegadamy co z tym zrobić.
#Poniżej zakomentowany kod usuwający wszystkie duplikaty.
"""
SELECT ID_DMC, CIRCUIT_NR, 
MAX(ASSIGMENT) AS ASSIGMENT, 
MAX(FLOW) AS FLOW, 
MAX(SET_POINT) AS SET_POINT,
MAX(START_DELAY) AS START_DELAY,
MAX(TEMP) AS TEMP,
MAX(WORKING_MODE) AS WORKING_MODE
FROM Z3DMC.ONI_CIRCUITS
GROUP BY ID_DMC, CIRCUIT_NR
ORDER BY ID_DMC
"""
#SELECT *
#FROM (
#  SELECT
#    t.*,
#    ROW_NUMBER() OVER (PARTITION BY ID_DMC ORDER BY ID_CIRCUIT DESC) AS rn
#  FROM
#    Z3DMC.ONI_CIRCUITS t
#) subquery
#WHERE rn = 1
]
username = 'dmcro'
password = 'ouZ9Rys7astDp4byjUvO'
dsn = 'vwpnwrud0000rac:1521/dmc.vwg'
dbhostname='vwpnwrud0000rac'
service_name='dmc.vwg'

In [33]:
try:
    sqlalchemy_engine="oracle+cx_oracle://"+username+":"+password+"@"+dbhostname+"/?service_name="+service_name
    engine = sqlalchemy.create_engine(sqlalchemy_engine, arraysize=1000)
    for table, query in zip(dbtables, querys):
        data.update({table: pd.read_sql(query, engine)})
except SQLAlchemyError as e:
    print(e)

<h1>Usuwanie zbędnych kolumn z tabel</h1>

In [34]:
dbtables = ['MEB_DGM', 'MEB_DMC', 'MEB_GROB', 'MEB_KO', 'MEB_KO_DGM', 'MEB_KS']
columns = [['timestamp', 'data_znakowania', 'data_odlania', 'metal_level', 'metal_pressure'],                                           #MEB_DGM
['timestamp', 'update_time','id_meb_containers', 'packed_time', 'first_packed_time', 'production_step', 'status_koncowy'],              #MEB_DMC
['id_meb_grob', 'shift_number', 'last_operation', 'timestamp', 'production_date', 'reworkrequested',                                    
'reworkdone', 'partcleaningisfinished', 'waitfortoolcheck', 'workingstep1', 'workingstep2', 'workingstep3', 'workingstep4', 'mms_ok'],  #MEB_GROB
['id_ko', 'data', 'timestamp', 'eks'],                                                                                                  #MEB_KO
['id_ko','data_odlania', 'timestamp', 'operator'],                                                                                      #MEB_KO_DGM
['id_ks', 'nrgniazda', 'liczbawystapien', 'nrformy', 'data', 'timestamp', 'gradedmc_max','gradedmc_aktualny']]                          #MEB_KS

for table, column in zip(dbtables, columns):
    data[table].drop(columns=column, inplace=True)
    #print(f'{table}: {data[table].columns}')

<h1>Łączenie tabel</h1>
Doklejać do MEB_DGM

In [35]:
# usuwanie znaków białych z DMC[MEB_DGM] i DMC_CASTING[MEB_DMC]
data['MEB_DMC'].dmc_casting = data['MEB_DMC']['dmc_casting'].str.strip()
data['MEB_DGM'].dmc = data['MEB_DGM']['dmc'].str.strip()

In [36]:
# usuwanie z meb_dmc wierszy z 'WORKPIECE NIO' w kodzie DMC
data['MEB_DMC'] = data['MEB_DMC'][~data['MEB_DMC']['dmc'].str.contains('WORKPIECE', case=False, na=False)]

In [37]:
# wybieranie rekordów dla MEB+ 
data['MEB_DGM'] = data['MEB_DGM'][(data['MEB_DGM']['nr_dgm'].between(8, 10)) & (data['MEB_DGM']['dmc'].apply(lambda x: len(x)) == 21)]
# usunięcie anomalii z MEB_DMC
data['MEB_DMC'] = data['MEB_DMC'][data['MEB_DMC']['dmc'].str[:3] == '0MH']

In [38]:
# łączę tabele MEB_KO i MEB_KO_DGM z tabelami MEB_KO_STREFA/RODZAJ

data['MEB_KO'] = data['MEB_KO'].merge(data['MEB_KO_STREFA'], left_on='nok_strefa', right_on='indeks', how='inner')
data['MEB_KO'].drop(columns=['indeks'], inplace=True)

data['MEB_KO'] = data['MEB_KO'].merge(data['MEB_KO_RODZAJ'], left_on='nok_rodzaj', right_on='indeks', how='inner')
data['MEB_KO'].drop(columns=['indeks'], inplace=True)

data['MEB_KO_DGM'] = data['MEB_KO_DGM'].merge(data['MEB_KO_STREFA'], left_on='nok_strefa', right_on='indeks', how='inner')
data['MEB_KO_DGM'].drop(columns=['indeks'], inplace=True)

data['MEB_KO_DGM'] = data['MEB_KO_DGM'].merge(data['MEB_KO_RODZAJ'], left_on='nok_rodzaj', right_on='indeks', how='inner')
data['MEB_KO_DGM'].drop(columns=['indeks'], inplace=True)

In [39]:
# łączę tabelę MEB_DMC z tabelą MEB_KO
data['MEB_DMC'] = data['MEB_DMC'].merge(data['MEB_KO'], on='id_dmc', how='left')
data['MEB_DMC'].drop(columns=['rn'], inplace=True)

In [40]:
# łączę tabelę MEB_DMC z tabelą MEB_GROB
data['MEB_DMC'] = data['MEB_DMC'].merge(data['MEB_GROB'], on='id_dmc', how='left')
data['MEB_DMC'].drop(columns=['rn'], inplace=True)

In [41]:
# łączę tabelę MEB_DMC z tabelą MEB_KS
data['MEB_DMC'] = data['MEB_DMC'].merge(data['MEB_KS'], on='id_dmc', how='left')
data['MEB_DMC'].drop(columns=['rn'], inplace=True)

In [42]:
# przygotowywuję tabelę ONI_CIRCUITS do połączenia 
oni_circuits = data['ONI_CIRCUITS'].pivot(index='id_dmc', columns='circuit_nr', values=['assigment', 'flow', 'set_point', 'start_delay', 'working_mode', 'temp'])
oni_circuits.columns = oni_circuits.columns.map('{0[0]}_{0[1]}'.format) 
oni_circuits.reset_index(inplace=True)

In [43]:
final_table = data['MEB_DGM'].copy()
final_table.drop(columns=['rn'], inplace=True)
# łączę tabelę MEB_DGM z tabelą MEB_KO_DGM
final_table = final_table.merge(data['MEB_KO_DGM'], left_on='id', right_on='id_dmc', how='left')
final_table.drop(columns=['rn'], inplace=True)
# łączę z tabelą MEB_DGM
final_table = final_table.merge(oni_circuits, left_on='id', right_on='id_dmc', how='inner')
final_table.drop(columns=['id_dmc_y'], inplace=True)
final_table.rename(columns={'id_dmc_x': 'id_dmc'}, inplace=True)
# łączę tabelę MEB_DMC z ONI_CIRCUITS
final_table = final_table.merge(data['MEB_DMC'], left_on='dmc', right_on='dmc_casting', how='left', suffixes=('_DGM', '_DMC'))

In [ ]:
duplicate_count_oni = final_table['dmc_DMC'].duplicated(keep=False).sum()
print(f"Number of rows with the same 'dmc' value: {duplicate_count_oni}")
final_table.drop(columns=['nok_strefa_DGM', 'nok_rodzaj_DGM', 'status_ko_DGM', 'kod_pola_DGM', 'rodzaj_uszkodzenia_DGM'], inplace=True)
final_table.rename(columns={'nok_strefa_DMC': 'nok_strefa', 'nok_rodzaj_DMC': 'nok_rodzaj', 
                            'status_ko_DMC': 'status_ko', 'kod_pola_DMC': 'kod_pola', 
                            'rodzaj_uszkodzenia_DMC': 'rodzaj_uszkodzenia'}, inplace=True)
                            
final_table.drop(index=final_table[(final_table['dmc_DGM'].duplicated(keep=False)) & (~final_table['dmc_casting'].isna())].index, inplace=True)
final_table.drop(columns = ['part_status'], inplace = True)

<h1>Usuwanie niepotrzebnych danych z tabeli koncowej</h1>

In [47]:
final_table = final_table[~final_table['status'].isin(['4', '5', '7', '8', '10', '11'])]
final_table['status'] = final_table['status'].replace(['3', '14'], '2')
final_table = final_table.loc[~final_table['status_ko'].isin([0, 106])]
final_table = final_table.loc[~final_table['statusszczelnosc'].isin([0, 3])]
final_table = final_table.loc[~final_table['statusdmc'].isin([0])]

In [ ]:
print(list(final_table.keys()))

In [ ]:
for i in range(1,29):
    print(final_table[f'ref_time_{i}'].value_counts())
    print('\n')

In [ ]:
for i in range(1,29):
    print(final_table[f'start_delay_{i}'].value_counts())
    print('\n')

In [ ]:
for i in range(1,29):
    print(final_table[f'working_mode_{i}'].value_counts())
    print('\n')

<h1>Tworzenie statusu końcowego</h1>

In [ ]:
# status:
#   1! - IO, 3! - NIO DGM, 4 - NIO DMC, 5 - NIO stolik pelen, 7 - NIO prasa, 8 - NIO znakowarka, 10 - NIO AT, 11 - NIO znakowarka zajęta, 14! - NIO vacuum
#   106 - Pomiary niszczące 110 - do pomiaru ręcznego, 
# rodzaj_kontroli
# status_koncowy - WYWALONY
# status_ko
# part_status - zostalo wywalone clkowicie bo mialo same 1 
# statusszczelnosc - 3 - do impregnacji - do wywalenia
# statusdmc
print(final_table['status'].value_counts().sort_index())
print(final_table['status_ko'].value_counts())
print(final_table['statusszczelnosc'].value_counts())
print(final_table['statusdmc'].value_counts())

In [48]:
final_table['our_final_status'] = final_table.apply(lambda row: max(int(row['status']), row['status_ko'], row['statusszczelnosc'], row['statusdmc']), axis=1)
print(final_table['our_final_status'].value_counts())

our_final_status
1.0    647583
2.0     27344
Name: count, dtype: int64


In [49]:
final_table.drop(columns=['status', 'status_ko', 'statusszczelnosc', 'statusdmc'], inplace=True)

<h1>Eliminacja NaN</h1>

In [64]:
# podmiana zer na NaNy, na następnym spotkaniu ustalimy co z tym zrobić
final_table.iloc[:, 28:196] = final_table[final_table.iloc[:, 28:196] != 0]
final_table.iloc[:, 28:196]

,assigment_1,assigment_2,assigment_3,assigment_4,assigment_5,assigment_6,assigment_7,assigment_8,assigment_9,assigment_10,assigment_11,assigment_12,assigment_13,assigment_14,assigment_15,assigment_16,assigment_17,assigment_18,assigment_19,assigment_20,assigment_21,assigment_22,assigment_23,assigment_24,assigment_25,assigment_26,assigment_27,assigment_28,flow_1,flow_2,flow_3,flow_4,flow_5,flow_6,flow_7,flow_8,flow_9,flow_10,flow_11,flow_12,flow_13,flow_14,flow_15,flow_16,flow_17,flow_18,flow_19,flow_20,flow_21,flow_22,flow_23,flow_24,flow_25,flow_26,flow_27,flow_28,set_point_1,set_point_2,set_point_3,set_point_4,set_point_5,set_point_6,set_point_7,set_point_8,set_point_9,set_point_10,set_point_11,set_point_12,set_point_13,set_point_14,set_point_15,set_point_16,set_point_17,set_point_18,set_point_19,set_point_20,set_point_21,set_point_22,set_point_23,set_point_24,set_point_25,set_point_26,set_point_27,set_point_28,start_delay_1,start_delay_2,start_delay_3,start_delay_4,start_delay_5,start_delay_6,start_delay_7,start_delay_8,start_delay_9,start_delay_10,start_delay_11,start_delay_12,start_delay_13,start_delay_14,start_delay_15,start_delay_16,start_delay_17,start_delay_18,start_delay_19,start_delay_20,start_delay_21,start_delay_22,start_delay_23,start_delay_24,start_delay_25,start_delay_26,start_delay_27,start_delay_28,working_mode_1,working_mode_2,working_mode_3,working_mode_4,working_mode_5,working_mode_6,working_mode_7,working_mode_8,working_mode_9,working_mode_10,working_mode_11,working_mode_12,working_mode_13,working_mode_14,working_mode_15,working_mode_16,working_mode_17,working_mode_18,working_mode_19,working_mode_20,working_mode_21,working_mode_22,working_mode_23,working_mode_24,working_mode_25,working_mode_26,working_mode_27,working_mode_28,temp_1,temp_2,temp_3,temp_4,temp_5,temp_6,temp_7,temp_8,temp_9,temp_10,temp_11,temp_12,temp_13,temp_14,temp_15,temp_16,temp_17,temp_18,temp_19,temp_20,temp_21,temp_22,temp_23,temp_24,temp_25,temp_26,temp_27,temp_28
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,17.0,16.0,17.0,17.0,17.0,17.0,17.0,17.0,16.0,17.0,17.0,2.0,16.0,17.0,17.0,17.0,17.0,17.0,17.0,17.0,17.0,17.0,16.0,17.0,17.0,17.0,17.0,17.0,NaN,NaN,11.1,14.5,10.8,14.3,7.8,9.1,NaN,2.5,9.3,9.8,NaN,10.3,11.5,11.6,11.6,11.6,12.6,11.5,10.6,3.7,NaN,11.1,8.5,15.0,11.6,13.5,10.0,2.0,NaN,15.0,NaN,25.0,NaN,20.0,2.0,15.0,25.0,25.0,2.0,20.0,20.0,25.0,17.0,5.0,NaN,35.0,15.0,5.0,2.0,15.0,NaN,15.0,NaN,15.0,7.0,15.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,5.0,12.0,2.0,5.0,2.0,2.0,5.0,2.0,2.0,2.0,2.0,21.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,3.0,2.0,3.0,2.0,3.0,2.0,2.0,2.0,2.0,NaN,2.0,2.0,2.0,2.0,2.0,2.0,3.0,2.0,2.0,2.0,2.0,2.0,3.0,2.0,3.0,2.0
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,17.0,16.0,17.0,17.0,17.0,17.0,17.0,17.0,16.0,16.0,17.0,17.0,16.0,17.0,17.0,17.0,17.0,17.0,17.0,17.0,17.0,17.0,16.0,17.0,17.0,17.0,17.0,17.0,NaN,NaN,11.8,15.1,12.1,16.7,7.6,11.6,NaN,NaN,10.1,11.0,NaN,11.6,11.1,13.3,11.5,11.8,12.8,12.8,14.3,4.6,NaN,13.5,9.1,14.3,12.3,15.6,10.0,2.0,10.0,15.0,NaN,25.0,NaN,20.0,2.0,5.0,25.0,25.0,2.0,20.0,20.0,25.0,17.0,5.0,NaN,35.0,15.0,5.0,2.0,15.0,NaN,15.0,25.0,15.0,7.0,15.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,20.0,2.0,5.0,12.0,2.0,5.0,2.0,2.0,5.0,2.0,2.0,2.0,2.0,21.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,3.0,2.0,3.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,3.0,2.0,2.0,2.0,2.0,2.0,3.0,2.0,2.0,2.0
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,17.0,16.0,17.0,17.0,17.0,17.0,17.0,17.0,16.0,17.0,17.0,2.0,16.0,17.0,17.0,17.0,17.0,17.0,17.0,17.0,17.0,17.0,16.0,17.0,17.0,17.0,17.0,17.0,NaN,NaN,11.1,14.5,10.8,14.3,7.8,9.1,NaN,2.7,9.3,9.6,NaN,10.3,11.3,11.1,11.6,11.8,12.6,11.5,10.6,3.7,NaN,11.1,8.5,14.8,11.6,13.5,10.0,2.0,NaN,15.0,NaN,25.0,NaN,20.0,2.0,15.0,25.0,25.0,2.0,20.0,20.0,25.0,17.0,5.0,NaN,35.0,15.0,5.0,2.0,15.0,NaN,15.0,NaN,15.0,7.0,15.0,2.0,2.0,2.0,2

<h1>Zebranie kolumn do dalszej dyskucji</h1>

In [21]:
# wyświetlenie kolumn z danymi numerycznymi
pd.set_option('display.max_columns', None)
final_table.select_dtypes(include=['number'])
# kolumny, które będą niepotrzebne: id, nr_dgm, status_ko, part_type, nrprogramu, id_dmc_DGM, id_dmc_DGM
# kolumny, o które trzeba zapytać produkcję: nachdruck_hub, anguss, max_press_kolbenhub, last_machine_number, last_pcf_number, machine_nr,
# cisnienie_tloka, rodzaj_kontroli, assigment

,id,nr_dgm,czas_fazy_1,czas_fazy_2,czas_fazy_3,max_predkosc,cisnienie_tloka,cisnienie_koncowe,nachdruck_hub,anguss,max_press_kolbenhub,temp_pieca,oni_temp_curr_f1,oni_temp_curr_f2,oni_temp_fore_f1,oni_temp_fore_f2,vds_air_pressure,vds_vac_hose1,vds_vac_hose2,vds_vac_tank,vds_vac_valve1,vds_vac_valve2,czas_taktu,id_dmc_DGM,rodzaj_kontroli,assigment_1,assigment_2,assigment_4,assigment_5,assigment_6,assigment_7,assigment_8,assigment_9,assigment_10,assigment_11,assigment_12,assigment_13,assigment_14,assigment_15,assigment_16,assigment_17,assigment_18,assigment_19,assigment_20,assigment_21,assigment_22,assigment_23,assigment_24,assigment_25,assigment_26,assigment_27,flow_2,flow_3,flow_4,flow_5,flow_6,flow_7,flow_8,flow_9,flow_10,flow_11,flow_12,flow_13,flow_14,flow_15,flow_16,flow_17,flow_18,flow_19,flow_20,flow_21,flow_22,flow_23,flow_24,flow_25,flow_26,flow_27,flow_28,set_point_1,set_point_2,set_point_3,set_point_4,set_point_5,set_point_6,set_point_7,set_point_8,set_point_9,set_point_10,set_point_11,set_point_12,set_point_13,set_point_14,set_point_15,set_point_16,set_point_17,set_point_18,set_point_19,set_point_20,set_point_21,set_point_22,set_point_23,set_point_24,set_point_25,set_point_26,set_point_27,set_point_28,start_delay_1,start_delay_2,start_delay_3,start_delay_4,start_delay_5,start_delay_6,start_delay_7,start_delay_8,start_delay_9,start_delay_10,start_delay_11,start_delay_12,start_delay_13,start_delay_14,start_delay_15,start_delay_16,start_delay_17,start_delay_18,start_delay_19,start_delay_20,start_delay_21,start_delay_22,start_delay_23,start_delay_24,start_delay_25,start_delay_26,start_delay_27,start_delay_28,working_mode_1,working_mode_2,working_mode_3,working_mode_4,working_mode_5,working_mode_6,working_mode_7,working_mode_8,working_mode_9,working_mode_10,working_mode_11,working_mode_12,working_mode_13,working_mode_14,working_mode_15,working_mode_16,working_mode_17,working_mode_18,working_mode_19,working_mode_20,working_mode_21,working_mode_22,working_mode_23,working_mode_24,working_mode_25,working_mode_26,working_mode_27,working_mode_28,temp_1,temp_2,temp_3,temp_4,temp_5,temp_6,temp_7,temp_8,temp_9,temp_10,temp_11,temp_12,temp_13,temp_14,temp_15,temp_16,temp_17,temp_18,temp_19,temp_20,temp_21,temp_22,temp_23,temp_24,temp_25,temp_26,temp_27,temp_28,id_dmc_DMC,nok_strefa,nok_rodzaj,part_type,last_machine_number,last_pcf_number,temp_workpiece,temp_hydraulics,pressure_pcf_1,pressure_pcf_2,pressure_pcf_3,machine_nr,cisnienie,przeciek,nrprogramu,temperaturatestu,our_final_status
0,315793,10.0,2277.0,90.0,45.0,6.30,5.0,282.0,27.0,35.0,2457.0,677.85,79.8,0.0,81.9,-1.0,5713.976,91.37731,90.79861,94.96528,171.2384,130.9028,94.0,NaN,NaN,17.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,16.0,17.0,17.0,17.0,17.0,17.0,17.0,16.0,17.0,17.0,2.0,16.0,17.0,17.0,17.0,17.0,17.0,17.0,17.0,17.0,17.0,16.0,17.0,17.0,17.0,17.0,17.0,NaN,NaN,11.1,14.5,10.8,14.3,7.8,9.1,NaN,2.5,9.3,9.8,NaN,10.3,11.5,11.6,11.6,11.6,12.6,11.5,10.6,3.7,NaN,11.1,8.5,15.0,11.6,13.5,10.0,2.0,NaN,15.0,NaN,25.0,NaN,20.0,2.0,15.0,25.0,25.0,2.0,20.0,20.0,25.0,17.0,5.0,NaN,35.0,15.0,5.0,2.0,15.0,NaN,15.0,NaN,15.0,7.0,15.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,5.0,12.0,2.0,5.0,2.0,2.0,5.0,2.0,2.0,2.0,2.0,21.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,3.0,2.0,3.0,2.0,3.0,2.0,2.0,2.0,2.0,NaN,2.0,2.0,2.0,2.0,2.0,2.0,3.0,2.0,2.0,2.0,2.0,2.0,3.0,2.0,3.0,2.0,0.0,0.0,1.0,3.0,6.0,27.400000,40.200001,101.0,104.0,90.0,3.0,1.061,0.000,1.0,31.199999,1.0
1,315795,9.0,2220.0,76.0,16.0,5.00,4.0,281.0,0.0,41.0,0.0,0.00,71.8,0.0,70.1,-1.0,4910.590,77.66204,73.61111,74.30556,381.5393,397.5116,85.0,NaN,NaN,17.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,16.0,17.0,17.0,17.0,17.0,17.0,17.0,16.0,16.0,17.0,17.0,16.0,17.0,17.0,17.0,17.0,17.0,17.0,17.0,17.0,17.0,16.0,17.0,17.0,17.0,17.0,17.0,NaN,NaN,11.8,15.1,12.1,16.7,7.6,11.6,NaN,NaN,10.1,11.0,NaN,11.6,11.1,13.3,11.5,11.8,12.8,12.8,14.3,4.6,NaN

In [22]:
# wyświetlanie danych kategorycznych
final_table.select_dtypes(include=['object'])
# kolumny do usunięcia: dmc_DGM, product_id, line_id, dmc_DMC, dmc_casting
# z jakiegoś powodu assigment 2,4,28 i flow 1 są danymi typu object - prawdopodobnie dlatego, że wszystkie wartości w kolumnach były równe 0 i zostały zmienione na NaN

,dmc_DGM,product_id,line_id,assigment_3,assigment_28,flow_1,dmc_DMC,dmc_casting,kod_pola,rodzaj_uszkodzenia
0,210623155611061147233,NaN,NaN,NaN,NaN,NaN,0MH301103DVW42106238970263102S612106231556,210623155611061147233,Brak,Brak
1,210623155710941036846,NaN,NaN,NaN,NaN,NaN,0MH301103DVW42106238976242102S412106231557,210623155710941036846,Brak,Brak
2,210623155811061147234,NaN,NaN,NaN,NaN,NaN,0MH301103DVW42106238970153102S612106231558,210623155811061147234,Brak,Brak
3,210623155911061147235,NaN,NaN,NaN,NaN,NaN,0MH301103DVW42106238969684102S612106231559,210623155911061147235,Brak,Brak
4,210623160111061147236,NaN,NaN,NaN,NaN,NaN,0MH301103DVW42106238969842102S612106231601,210623160111061147236,Brak,Brak
...,...,...,...,...,...,...,...,...,...,...
707521,230918113021052170478,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
707522,230918113121052170479,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
707523,230918113321052170480,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
707524,230918113421052170481,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [68]:
# DO DYSKUCJI!
final_table.drop(columns=['id', 'nr_dgm', 'part_type', 'nrprogramu', 'id_dmc_DGM', 'id_dmc_DGM', 
                          'dmc_DGM', 'product_id', 'line_id', 'dmc_DMC', 'dmc_casting'
                          'flow_1', 'flow_3', 'flow_5', ], inplace=True)

<h1>Normalizacja danych</h1>

In [25]:
# Przekształcenie danych kategorycznych
final_table['kod_pola'].value_counts()

kod_pola
Brak    610082
R10       6077
R09       1247
R03       1036
R08        813
         ...  
S07          2
R45          1
R44          1
R30          1
S31          1
Name: count, Length: 66, dtype: int64

In [28]:
final_table = pd.get_dummies(final_table, columns=['kod_pola', 'rodzaj_uszkodzenia'], dummy_na=True, dtype='uint8')

,id,dmc_DGM,nr_dgm,czas_fazy_1,czas_fazy_2,czas_fazy_3,max_predkosc,cisnienie_tloka,cisnienie_koncowe,nachdruck_hub,anguss,max_press_kolbenhub,temp_pieca,oni_temp_curr_f1,oni_temp_curr_f2,oni_temp_fore_f1,oni_temp_fore_f2,vds_air_pressure,vds_vac_hose1,vds_vac_hose2,vds_vac_tank,vds_vac_valve1,vds_vac_valve2,czas_taktu,id_dmc_DGM,rodzaj_kontroli,product_id,line_id,assigment_1,assigment_2,assigment_3,assigment_4,assigment_5,assigment_6,assigment_7,assigment_8,assigment_9,assigment_10,assigment_11,assigment_12,assigment_13,assigment_14,assigment_15,assigment_16,assigment_17,assigment_18,assigment_19,assigment_20,assigment_21,assigment_22,assigment_23,assigment_24,assigment_25,assigment_26,assigment_27,assigment_28,flow_1,flow_2,flow_3,flow_4,flow_5,flow_6,flow_7,flow_8,flow_9,flow_10,flow_11,flow_12,flow_13,flow_14,flow_15,flow_16,flow_17,flow_18,flow_19,flow_20,flow_21,flow_22,flow_23,flow_24,flow_25,flow_26,flow_27,flow_28,set_point_1,set_point_2,set_point_3,set_point_4,set_point_5,set_point_6,set_point_7,set_point_8,set_point_9,set_point_10,set_point_11,set_point_12,set_point_13,set_point_14,set_point_15,set_point_16,set_point_17,set_point_18,set_point_19,set_point_20,set_point_21,set_point_22,set_point_23,set_point_24,set_point_25,set_point_26,set_point_27,set_point_28,start_delay_1,start_delay_2,start_delay_3,start_delay_4,start_delay_5,start_delay_6,start_delay_7,start_delay_8,start_delay_9,start_delay_10,start_delay_11,start_delay_12,start_delay_13,start_delay_14,start_delay_15,start_delay_16,start_delay_17,start_delay_18,start_delay_19,start_delay_20,start_delay_21,start_delay_22,start_delay_23,start_delay_24,start_delay_25,start_delay_26,start_delay_27,start_delay_28,working_mode_1,working_mode_2,working_mode_3,working_mode_4,working_mode_5,working_mode_6,working_mode_7,working_mode_8,working_mode_9,working_mode_10,working_mode_11,working_mode_12,working_mode_13,working_mode_14,working_mode_15,working_mode_16,working_mode_17,working_mode_18,working_mode_19,working_mode_20,working_mode_21,working_mode_22,working_mode_23,working_mode_24,working_mode_25,working_mode_26,working_mode_27,working_mode_28,temp_1,temp_2,temp_3,temp_4,temp_5,temp_6,temp_7,temp_8,temp_9,temp_10,temp_11,temp_12,temp_13,temp_14,temp_15,temp_16,temp_17,temp_18,temp_19,temp_20,temp_21,temp_22,temp_23,temp_24,temp_25,temp_26,temp_27,temp_28,id_dmc_DMC,dmc_DMC,dmc_casting,nok_strefa,nok_rodzaj,part_type,last_machine_number,last_pcf_number,temp_workpiece,temp_hydraulics,pressure_pcf_1,pressure_pcf_2,pressure_pcf_3,machine_nr,cisnienie,przeciek,nrprogramu,temperaturatestu,our_final_status,kod_pola_B01,kod_pola_B02,kod_pola_B04,kod_pola_B07,kod_pola_B56,kod_pola_Brak,kod_pola_R00,kod_pola_R02,kod_pola_R03,kod_pola_R04,kod_pola_R05,kod_pola_R06,kod_pola_R07,kod_pola_R08,kod_pola_R09,kod_pola_R10,kod_pola_R11,kod_pola_R12,kod_pola_R13,kod_pola_R14,kod_pola_R15,kod_pola_R16,kod_pola_R17,kod_pola_R18,kod_pola_R19,kod_pola_R20,kod_pola_R21,kod_pola_R22,kod_pola_R26,kod_pola_R27,kod_pola_R28,kod_pola_R30,kod_pola_R31,kod_pola_R41,kod_pola_R42,kod_pola_R43,kod_pola_R44,kod_pola_R45,kod_pola_R56,kod_pola_S00,kod_pola_S01,kod_pola_S02,kod_pola_S03,kod_pola_S04,kod_pola_S05,kod_pola_S07,kod_pola_S08,kod_pola_S09,kod_pola_S10,kod_pola_S11,kod_pola_S12,kod_pola_S13,kod_pola_S14,kod_pola_S15,kod_pola_S16,kod_pola_S17,kod_pola_S18,kod_pola_S19,kod_pola_S20,kod_pola_S21,kod_pola_S22,kod_pola_S26,kod_pola_S27,kod_pola_S31,kod_pola_S42,kod_pola_S56,kod_pola_nan,rodzaj_uszkodzenia_Brak,rodzaj_uszkodzenia_Mechaniczne - obróbka MF,rodzaj_uszkodzenia_Mechaniczne - odlewnicze,rodzaj_uszkodzenia_Odlewnicze - brak naddatku,rodzaj_uszkodzenia_Odlewnicze - porowatości,rodzaj_uszkodzenia_Odlewnicze - wtrącenia,rodzaj_uszkodzenia_Wymiarowe - obróbka MF,rodzaj_uszkodzenia_Wymiarowe - odlewnicze,rodzaj_uszkodzenia_nan
0,315793,210623155611061147233,10.0,2277.0,90.0,45.0,6.30,5.0,282.0,27.0,35.0,2457.0,677.85,79.8,0.0,81.9,-1.0,5713.976,91.37731,90.79861,94.96528,171.2384,130.9028,94.0,

In [ ]:
# Normalizacja danych numerycznych
